In [ ]:
#import necessary libraries

import scanpy as sc
import numpy as np
import pandas as pd

#import plotting
import matplotlib.pyplot as plt
import seaborn as sns

print("libraries imported successfully")

In [ ]:
#load data

adata = sc.datasets.pbmc3k()
#this command tells scanpy to download and load the pbmc3k dataset

print(adata.n_obs)
print(f'Data loaded! Original number of cells: {adata.n_obs}')

In [ ]:
print(f'staring with {adata.n_obs} cells.')
print(adata)

In [ ]:
# .var holds the metadata about the genes (variables) 
# .head() shows the first 5 rows 

adata.var.head()

In [ ]:
# .obs holds the metadata about the cells (observations) 
# .head() shows the first 5 rows 

adata.obs.head()

In [ ]:
#----QC and filtering---- 

#finding mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('MT-')

#QC metrics calculation
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
print("QC metrics calculated! Here is the updated table:")
adata.obs.head()

In [ ]:
#----QC and filtering---- 

#violin plot to show the probability density of the data at different values

#pl is short for 'plotting'
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], 
             jitter= 0.4, multi_panel=True)

# jitter=0.4 to add random noise to indiviual points so I can see them better and wont fall on a single vertical line 
# multi_panel=True tells scanpy to arrange the 3 plots neatly next to each other

plt.show

In [ ]:
#----QC and filtering---- 

print(f'Number of cells before filtering: {adata.n_obs}')
adata = adata[adata.obs.n_genes_by_counts > 200, :]
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]
print(f'Number of cells after filtering: {adata.n_obs}')

In [ ]:
#---Normalization, HVG Selection, & Scaling----

#normalize and log transform
#each cell to have a total of 10k molecules to correct for differences in library size(sequencing depth) per cell
#pp- preprocessing, this function goes throgh each cell, scales its counts up/down so that the total count (target_sum) for each cell is 10k
sc.pp.normalize_total(adata, target_sum=1e4)
#by taking the natural log of the data, the variance will become more stable across genes with different expression levels
#takes the data from previous step and applies a log transformation to all expression values; log1p: to avoid errors from taking the log of zero
sc.pp.log1p(adata)
print("normalization and log-transformation complete.")

#****saving a backup of the full data before we filter it****
adata.raw=adata
print('created a .raw backup of the full dataset.')

In [ ]:
#---Normalization, HVG Selection, & Scaling----

#we need to find genes whose expresssion levels change alot across different cells- HVGs - they hold the most bioloigcal information

#identifies HVGs
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
print(f'Number of highly variable genes found: {adata.var.highly_variable.sum()}')

#plot results
sc.pl.highly_variable_genes(adata)
plt.show()

In [ ]:
#---Normalization, HVG Selection, & Scaling---

# to check if the correct data was stored
adata.var['highly_variable'].value_counts()

In [ ]:
#---Normalization, HVG Selection, & Scaling----

original_var_count = adata.n_vars
print(f'original number of genes:{original_var_count}')

#subsetting the AnnData object to only the HVG
adata = adata[:, adata.var.highly_variable]
print(f'subsetting to {adata.n_vars} highly variable genes.')

#scaling the data
sc.pp.scale(adata, max_value=10)
print('\nScaling complete.')

In [ ]:
#----PCA, Clustering, UMAP----- 

#run the PCA calculation, use the tl for tools function to do the math
sc.tl.pca(adata, svd_solver='arpack')

#plot the PCA analysis, and color the cells by the expression of the CST3 gene which is a monocyte marker
sc.pl.pca(adata, color='CST3')
plt.show()

In [ ]:
#----PCA, Clustering, UMAP----- 

#Elbow plot
#plot the variance explained by each PC to find the "elbow" (shows how much variance each PC explains to the point (elbow) where adding another PC doesnt give us new info)
sc.pl.pca_variance_ratio(adata, log=True)
plt.show()

#the amount of PCs is x-axis and vairance levels is y-axis

In [ ]:
#----PCA, Clustering, UMAP----- 

#neighborhood graph and clustering 

#using 10 PCs decided from the PC plot
n_pcs_to_use = 10 
print(f'Using {n_pcs_to_use} Principal Components for the next steps.')

#computing the neighborhood graph to find the nearest neighbors for each cell in the PCA space
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=n_pcs_to_use)

#cluster cells using the leiden algorithm to find the communities of cells in the neighborhood graph 
sc.tl.leiden(adata)

print(f'\nNeighborhood graph computed and cells have been clustered.')
print(f"The cluster labels are now stored in 'adata.obs['leiden']'.")

#first few rows of the observation metadata with a new leiden column with the cluster numbers
adata.obs.head()

In [ ]:
#----PCA, Clustering, UMAP----- 

#visualizing clusters with UMAP
sc.tl.umap(adata)

#plotting UMAP colored by cluster
sc.pl.umap(adata, color='leiden')
plt.show()

In [ ]:
#----Marker gene annotation----- 

# finding marker genes for each cluster compared to other clusters
sc.tl.rank_genes_groups(adata, groupby='leiden', method='t-test') #statistical test - t-test to find genes that are expressed differently in each leiden cluster and then sort into the adata object

#plot the top marker genes for each cluster 
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False) #top 25 genes
plt.show()

print('done. ranked gene lists are now stored in the adata object.')

In [ ]:
#----Marker gene annotation----- 

#define a specific list of key marker genes to visualize
#these genes are famous markers for the main immune cell types
marker_genes_to_plot = ['IL7R', 'CD3D',  #T-cell markers
                        'MS4A1', 'CD79A', #B-cell markers
                        'LYZ', 'CST3',   #Monocyte markers
                        'NKG7', 'GNLY',   #NK cell markers
                        'PPBP']           #Platelet marker

print("A specific list of marker genes has been created.")


#creating a dotplot and using the raw function to find marker genes
sc.pl.dotplot(adata, marker_genes_to_plot, groupby='leiden', use_raw=True)
plt.show()

In [ ]:
#----Marker gene annotation----- 

#visualizing the markers on UMAP using the marker genes
sc.pl.umap(adata, color=['CD3D', 'MS4A1', 'LYZ', 'NKG7', 'IL7R', 'CD79A', 'CST3', 'GNLY', 'PPBP'], use_raw=True)
plt.show()

In [ ]:
#---Setting up Biological Interpretation----

#plotting UMAP with Leiden cluster on plot to study and identify the islands
print('Reference UMAP with Leiden cluster IDs:')
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='Leiden Clusters')
plt.show()

In [ ]:
#---Setting up Biological Interpretation----

#visualizing a panel of T-cell Markers
print("Visualizing T-cell subtype markers...")
sc.pl.umap(adata, color=['CD3D', 'CD4', 'CD8A'], use_raw=True)
plt.show()

In [ ]:
#for the clusters that cannot be identified via dotplot and umap...

import pandas as pd

print('top 5 marker genes for each cluster:')
print(pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5))

In [ ]:
#for the clusters that cannot be identified via dotplot and umap
#replace '---' with the top gene you found for your cluster to confirm and view visually
#sc.pl.umap(adata, color=['--'], use_raw=True)
#plt.show

In [ ]:
#---Biological Interpretation----

#creating a dictionary to map the cluster numbers to the cells types I identified 
final_cluster_to_celltype={'0': 'CD4 T-Cells',
 '1': 'B Cells',
 '2': 'Monocytes',
 '3': 'Stressed/Ribosomal proteins',
 '4': 'Stressed/Ribosomal proteins',
 '5': 'Monocytes',
 '6': 'NK Cells',
 '7': 'CD8 T-Cells',
 '8': 'Naive T-Cells',
 '9': 'Stressed/Ribosomal proteins',
 '10': 'Monocytes',
 '11': 'Stressed/Ribosomal proteins',
 '12': 'Doublets',
 '13': 'Dendritic Cells',
 '14': 'Platelets'}

#creating a new column in data table called 'cell type' and convert leiden to string to be mapped by dictionary
adata.obs['cell_type'] = adata.obs['leiden'].astype(str).map(final_cluster_to_celltype).astype('category')

#final annotated UMAP
print('\nDisplying the final annotated UMAP plot:')
sc.pl.umap(adata, color='cell_type', title='Final PBMC Cell Type Annotations', legend_loc='right margin' )
plt.show()

In [ ]:
#-----Done!-----